# Fantasy Footbal VBD Auction Creator and Optimizer
### DATA NOTE: SOurced via FantasyPros.com

1. Creates Value-based drafting values from 'Points above replacement' concept
2. Determines optimal split per cost on any position (eg. 80 to RB = 60 RB1; 20 RB2)
3. Runs optimal team splits (% on each position)

Remaining issues:
- QB values are wonky
    - Normalize for prior year values?
    - Manually adjust upwards?
- Optimizating of team splits has too many calculations
    - Narrow search ranges
    - Increase percentage increment?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df_raw_stats = pd.read_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2020 Data\Combined_data_2020.csv')

In [3]:
df_raw_stats = df_raw_stats.fillna(0)
df_raw_stats['Position_Rank'] = df_raw_stats.groupby('Pos')['Exp_PTS'].rank(ascending=False)


### Define league settings

In [30]:
teams = 12
buyin = 200
qb_start, qb_bench = 2, 0.5
rb_start, rb_bench = 2.5, .5
wr_start, wr_bench = 2.5, 2
te_start, te_bench = 1, .25
bench_size = 5 ### actual 6, but calculated to optimize for 2 flex. 
d_start = 1
k_start = 1
rb_waiver = int(teams*(rb_start+rb_bench)+1)
wr_waiver = int(teams*(wr_start+wr_bench)+1)
#qb_waiver = int(teams*(qb_start+qb_bench)+1)
qb_waiver = 28
te_waiver = int(teams*(te_start+te_bench)+1)
op_waiver = int(teams*14+1)

## Run assumptions

In [31]:
starters = .96
### position multipliers based on prior years



# Replacement values

In [32]:
# Find replacement-level RB.
#### Replacement = (rb_start+rb_bench)*teams+1
#### Look up points of replacement-level by pos. 
df_rb = df_raw_stats.where(df_raw_stats['Pos']=='RB')
repl_rb=list(df_rb.where(df_rb['Position_Rank']==int(rb_waiver)).dropna().iloc[0])[12]

df_qb = df_raw_stats.where(df_raw_stats['Pos']=='QB')
repl_qb=list(df_qb.where(df_qb['Position_Rank']==int(qb_waiver)).dropna().iloc[0])[12]

df_wr = df_raw_stats.where(df_raw_stats['Pos']=='WR')
repl_wr=list(df_wr.where(df_wr['Position_Rank']==int(wr_waiver)).dropna().iloc[0])[12]

df_te = df_raw_stats.where(df_raw_stats['Pos']=='TE')
repl_te=list(df_te.where(df_te['Position_Rank']==int(te_waiver)).dropna().iloc[0])[12]

In [33]:
df_rb['PAR'] = np.max(df_rb['Exp_PTS']-0,0)
df_rb['PAR'] =df_rb.apply(lambda row: df_rb['Exp_PTS'] -repl_rb,axis=0)
df_rb.loc[df_rb['PAR']<0,'PAR'] = 0

df_wr['PAR'] = np.max(df_wr['Exp_PTS']-0,0)
df_wr['PAR'] =df_wr.apply(lambda row: df_wr['Exp_PTS'] -repl_wr,axis=0)
df_wr.loc[df_wr['PAR']<0,'PAR'] = 0

df_qb['PAR'] = np.max(df_qb['Exp_PTS']-0,0)
df_qb['PAR'] =df_qb.apply(lambda row: df_qb['Exp_PTS'] -repl_qb,axis=0)
df_qb.loc[df_qb['PAR']<0,'PAR'] = 0

df_te['PAR'] = np.max(df_te['Exp_PTS']-0,0)
df_te['PAR'] =df_te.apply(lambda row: df_te['Exp_PTS'] -repl_te,axis=0)
df_te.loc[df_te['PAR']<0,'PAR'] = 0


In [35]:
df_op2 = df_raw_stats[(df_raw_stats['Pos']!='DEF')&(df_raw_stats['Pos']!='K')]
repl_op = df_op2['Exp_PTS'].values[op_waiver]
df_op2['PAR'] = df_op2['Exp_PTS'] - repl_op
df_op2.loc[df_op2['PAR']<0,'PAR'] = 0
df_op2.head()

C:\Users\stefa\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\stefa\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Pos,Player,Team,Bye,Comp,Att,Yard,TD,INT,Att.1,Yard.1,TD.1,Exp_PTS,Hi_PTS,Lo_PTS,Position_Rank,PAR
0,QB,Lamar Jackson,BAL,8,282.0,440.0,"3,255",30,12.0,165,988,6,355.0,355.0,355.0,1.0,212.9
1,QB,Patrick Mahomes,KC,10,373.0,566.0,"4,582",34,9.0,56,229,2,330.2,330.2,330.2,2.0,188.1
2,QB,Dak Prescott,DAL,10,387.0,596.0,"4,530",28,12.0,71,304,4,323.6,323.6,323.6,3.0,181.5
3,QB,Russell Wilson,SEA,6,321.0,494.0,"3,950",30,8.0,74,361,3,310.1,310.1,310.1,4.0,168.0
4,QB,Kyler Murray,ARI,8,374.0,585.0,"4,154",25,13.0,97,513,3,301.5,301.5,301.5,5.0,159.4


In [36]:
df_op = df_rb.append(df_qb, ignore_index=True)
df_op = df_op.append(df_te, ignore_index=True)
df_op = df_op.append(df_wr, ignore_index=True)
df_op.dropna()
total_par = np.sum(df_op['PAR'])
value_per_pt = teams * (buyin*starters) / total_par

In [37]:
print(value_per_pt)
teams*buyin*starters/df_op2['PAR'].sum()

0.37177480515708455


0.44951712028094815

In [38]:
df_op['Auction Value'] = np.max(df_op['PAR'],0)
df_op['Auction Value'] = df_op.apply(lambda row: df_op['PAR'] * value_per_pt,axis=0)
df_op['Downside'] = (df_op['Lo_PTS']-df_op['Exp_PTS'])*value_per_pt
df_op['Upside'] = (df_op['Hi_PTS']-df_op['Exp_PTS'])*value_per_pt
df_op.dropna()
df_op.sort_values(by=['Auction Value'], ascending=False)
df_pres = df_op[['Pos','Player','Team','Auction Value','Upside','Downside','Exp_PTS','Hi_PTS','Lo_PTS']]
df_pres = df_pres.sort_values(by=['Auction Value','Exp_PTS'], ascending=False)



In [39]:
#df_pres.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2020 Data\Auction_Value_2020.csv')

In [40]:
df_pres = df_pres[df_pres['Exp_PTS']>0].dropna()
df_pres[df_pres['Pos']=='QB'][:30]

,Pos,Player,Team,Auction Value,Upside,Downside,Exp_PTS,Hi_PTS,Lo_PTS
364,QB,Lamar Jackson,BAL,53.535572,0.0,0.0,355.0,355.0,355.0
365,QB,Patrick Mahomes,KC,44.315557,0.0,0.0,330.2,330.2,330.2
366,QB,Dak Prescott,DAL,41.861843,0.0,0.0,323.6,323.6,323.6
367,QB,Russell Wilson,SEA,36.842883,0.0,0.0,310.1,310.1,310.1
368,QB,Kyler Murray,ARI,33.645620,0.0,0.0,301.5,301.5,301.5
369,QB,Deshaun Watson,HOU,32.939248,0.0,0.0,299.6,299.6,299.6
370,QB,Josh Allen,BUF,31.749568,0.0,0.0,296.4,296.4,296.4
371,QB,Carson Wentz,PHI,28.552305,0.0,0.0,287.8,287.8,287.8
372,QB,Matt Ryan,ATL,28.180530,0.0,0.0,286.8,286.8,286.8
373,QB,Drew Brees,NO,26.321656,0.0,0.0,281.8,281.8,281.8


### Sanity Check on Auction Values
- Compare values for top positions vs. prior years.

In [41]:
df_2019 = pd.read_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2020 Data\Actual vs. Predicted 2019.csv')
df_2019.head()

,Pos,PosRank,Player,Team,Auction Value,Upside,Downside,Exp_PTS,Hi_PTS,Lo_PTS,Actual,Actual - Pred
0,QB,1,Patrick Mahomes,KC,41.776675,6.997519,-2.828784,339.6,363.1,330.1,41,-0.8
1,QB,2,Deshaun Watson,HOU,35.821340,5.985112,-3.126551,319.6,339.7,309.1,41,5.2
2,QB,3,Baker Mayfield,CLE,26.024814,6.908189,-2.888337,286.7,309.9,277.0,40,14.0
3,QB,4,Lamar Jackson,BAL,24.655087,9.945409,-7.027295,282.1,315.5,258.5,39,14.3
4,QB,5,Aaron Rodgers,GB,33.439206,8.099256,-3.870968,311.6,338.8,298.6,36,2.6


In [42]:
df_2019[df_2019['Pos']=='RB']

,Pos,PosRank,Player,Team,Auction Value,Upside,Downside,Exp_PTS,Hi_PTS,Lo_PTS,Actual,Actual - Pred
30,RB,1,David Johnson,ARI,32.337469,9.588089,-5.657568,214.4,246.6,195.4,56,23.7
31,RB,2,Saquon Barkley,NYG,49.995037,7.176179,-5.210918,273.7,297.8,256.2,55,5.0
32,RB,3,Ezekiel Elliott,DAL,46.183623,10.630273,-8.754342,260.9,296.6,231.5,54,7.8
33,RB,4,Christian McCaffrey,CAR,43.890819,9.766749,-5.836228,253.2,286.0,233.6,53,9.1
34,RB,5,Alvin Kamara,NO,42.729529,5.270471,-1.369727,249.3,267.0,244.7,53,10.3
...,...,...,...,...,...,...,...,...,...,...,...,...
86,RB,57,Damien Harris,NE,0.000000,9.588089,-5.717122,63.2,95.4,44.0,1,1.0
87,RB,58,Alexander Mattison,MIN,0.000000,12.595534,-6.193548,53.0,95.3,32.2,1,1.0
88,RB,59,Justice Hill,BAL,0.000000,12.625310,-5.062035,46.5,88.9,29.5,1,1.0
89,RB,60,Darwin Thompson,KC,0.000000,14.531017,-7.414392,39.2,88.0,14.3,1,1.0


In [59]:
tier_list = [3,6,12,24,140]
pos_list = ['QB', 'RB', 'WR', 'TE']

df_comp = pd.DataFrame()
for pos in pos_list:
    for tier in tier_list: 
        cur_proj = round(df_pres[df_pres['Pos']==pos]['Auction Value'].values[:tier].mean(),0)
        last_actual = round(df_2019[df_2019['Pos']==pos]['Actual'].values[:tier].mean(),0) 
        df_temp = pd.DataFrame(data=({'Pos':pos,
                                      'tier':tier, 
                                      'Proj':cur_proj, 
                                      'Last_act': last_actual}), index=[0])
        df_comp = df_comp.append(df_temp)
df_comp['ratio'] = df_comp['Last_act'] / df_comp['Proj'] 

In [60]:
pos_list = df_2019[df_2019['Pos']=='RB']['Auction Value'].values
pos_list = np.sort(pos_list)[::-1]
pos_list

array([49.99503722, 46.18362283, 43.89081886, 42.72952854, 32.33746898,
       31.83126551, 31.23573201, 30.66997519, 29.86600496, 29.18114144,
       27.81141439, 26.62034739, 26.44168734, 25.9057072 , 23.34491315,
       22.54094293, 21.26054591, 20.87344913, 20.33746898, 19.98014888,
       19.86104218, 19.05707196, 18.37220844, 15.03722084, 13.9057072 ,
       13.3101737 , 13.191067  ,  8.87344913,  7.02729529,  5.35980149,
        4.01985112,  3.27543424,  3.12655087,  2.67990074,  0.74441687,
        0.62531017,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ])

In [61]:
def adjuster(pos, pos_rank):
     return df_comp[(df_comp['Pos']==pos)&(df_comp['tier']>=pos_rank)]['ratio'].values[0]
    
def last_year_act(pos, pos_rank):
    try:
        act = df_2019[(df_2019['Pos']==pos)&(df_2019['PosRank']==pos_rank)]['Actual'].values[0]
    except:
        act = 0
    return act

def last_year_pred(pos, pos_rank):
    pos_list = df_2019[df_2019['Pos']==pos]['Auction Value'].values
    pos_list = np.sort(pos_list)[::-1]
    try:
        pred = pos_list[int(pos_rank)-1]
    except: 
        pred = 0 
    return pred

In [62]:
df_comp

,Pos,tier,Proj,Last_act,ratio
0,QB,3,47.0,41.0,0.872340
0,QB,6,41.0,39.0,0.951220
0,QB,12,34.0,36.0,1.058824
0,QB,24,23.0,28.0,1.217391
0,QB,140,11.0,22.0,2.000000
0,RB,3,56.0,55.0,0.982143
0,RB,6,51.0,53.0,1.039216
0,RB,12,42.0,44.0,1.047619
0,RB,24,32.0,33.0,1.031250
0,RB,140,8.0,15.0,1.875000


In [63]:
df_pres['pos_rank'] = df_pres.groupby('Pos')['Exp_PTS'].rank(axis=0, ascending=False, method='min')
df_pres['expected_auction'] = np.minimum(2,np.vectorize(adjuster)(df_pres['Pos'], df_pres['pos_rank']))*df_pres['Auction Value']
df_pres['last_year_posrank_act'] = np.vectorize(last_year_act)(df_pres['Pos'], df_pres['pos_rank'])
df_pres['last_year_posrank_pred'] = np.vectorize(last_year_pred)(df_pres['Pos'], df_pres['pos_rank'])
df_pres.head(10)


,Pos,Player,Team,Auction Value,Upside,Downside,Exp_PTS,Hi_PTS,Lo_PTS,pos_rank,expected_auction,last_year_posrank_act,last_year_posrank_pred
50,RB,Christian McCaffrey,CAR,65.543898,0.0,0.0,286.5,286.5,286.5,1.0,64.373471,56,49.995037
364,QB,Lamar Jackson,BAL,53.535572,0.0,0.0,355.0,355.0,355.0,1.0,46.701244,41,41.776675
51,RB,Dalvin Cook,MIN,51.639520,0.0,0.0,249.1,249.1,249.1,2.0,50.717386,55,46.183623
52,RB,Ezekiel Elliott,DAL,51.304923,0.0,0.0,248.2,248.2,248.2,3.0,50.388764,54,43.890819
53,RB,Saquon Barkley,NYG,50.375486,0.0,0.0,245.7,245.7,245.7,4.0,52.350995,53,42.729529
54,RB,Derrick Henry,TEN,50.301131,0.0,0.0,245.5,245.5,245.5,5.0,52.273725,53,32.337469
365,QB,Patrick Mahomes,KC,44.315557,0.0,0.0,330.2,330.2,330.2,2.0,38.658252,41,35.821340
366,QB,Dak Prescott,DAL,41.861843,0.0,0.0,323.6,323.6,323.6,3.0,36.517778,40,33.439206
1242,WR,Michael Thomas,NO,41.490068,0.0,0.0,201.8,201.8,201.8,1.0,45.857444,43,35.255583
1243,WR,Davante Adams,GB,37.921030,0.0,0.0,192.2,192.2,192.2,2.0,41.912718,42,35.047146


In [64]:
df_pres = df_pres[['Pos','pos_rank','Player','Team','Exp_PTS','Auction Value','expected_auction','last_year_posrank_act','last_year_posrank_pred']]
df_pres

,Pos,pos_rank,Player,Team,Exp_PTS,Auction Value,expected_auction,last_year_posrank_act,last_year_posrank_pred
50,RB,1.0,Christian McCaffrey,CAR,286.5,65.543898,64.373471,56,49.995037
364,QB,1.0,Lamar Jackson,BAL,355.0,53.535572,46.701244,41,41.776675
51,RB,2.0,Dalvin Cook,MIN,249.1,51.639520,50.717386,55,46.183623
52,RB,3.0,Ezekiel Elliott,DAL,248.2,51.304923,50.388764,54,43.890819
53,RB,4.0,Saquon Barkley,NYG,245.7,50.375486,52.350995,53,42.729529
...,...,...,...,...,...,...,...,...,...
411,QB,48.0,Jeff Driskel,DEN,11.2,0.000000,0.000000,0,0.000000
412,QB,49.0,AJ McCarron,HOU,11.0,0.000000,0.000000,0,0.000000
413,QB,50.0,Case Keenum,CLE,10.7,0.000000,0.000000,0,0.000000
148,RB,99.0,Peyton Barber,WAS,10.2,0.000000,0.000000,0,0.000000


In [66]:
df_pres.to_csv(r'C:\Users\stefa\OneDrive\Documents\Analysis Projects\Fantasy Modeling\Football\2020 Data\Auction_Value_2020.csv')